In [ ]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
class CategoricalEncoder:
    
    """
    It also changes the data type to int
    """
    
    def __init__(self, cols, method = 'classic'):
        self.COLUMNS = cols
        self.METHOD = method
        self.encoder = {}
        self.inverse_encoder = {}
        
    def fit(self, X, y=None):
        X = X.copy()
        if self.METHOD == 'classic':
            for col in self.COLUMNS:
                self.encoder[col] = {}
                self.inverse_encoder[col] = {}
                unique_values = X[col].unique()
                for i, val in enumerate(unique_values):
                    self.encoder[col][val] = i
                    self.inverse_encoder[col][i] = val
                    
        if self.METHOD == 'mean':
            for col in self.COLUMNS:
                self.encoder[col] = {}
                self.inverse_encoder[col] = {}
                unique_values = X[col].unique()
                for val in unique_values:
                    tmp = X[col][X[col] == val]
                    mean = y.loc[tmp.index, 'P80'].mean()
                    self.encoder[col][val] = mean
                    self.inverse_encoder[col][mean] = val
                self.encoder['NaN'] = y['P80'].mean()
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in self.COLUMNS:
            X[col] = X[col].map(self.encoder[col]).fillna(self.encoder['NaN'])
        return X
    
    def inverse_transform(self, X):
        X = X.copy()
        for col in self.COLUMNS:
            X[col] = X[col].replace(self.inverse_encoder[col])
        return X
    

In [ ]:
with open('models/CONF.pickle', 'rb') as handle:
    CONF = pickle.load(handle)
    
with open('models/encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)

In [ ]:
df_historic = pd.read_csv('data/data_fixed.csv', sep = ';')
df_historic = df_historic[df_historic['Tipo Explosivo'] != 'M']
df_historic = df_historic.dropna()

In [ ]:
"""restricciones:
    -100 < FC < 1000
    -tiempo_1 < tiempo_2
    -Diametro, Fraccion, dist min, dist max
    12.25 == 1/4 -> 5.5 - 15 m
    10.625 == 10 5/8 -> 5.5 - 15 m
    9.875 == 9 7/8 -> 5 - 15m
    7.875 == 7 7/8 -> 4 - 15m
    6.5 == 6 1/2 -> 4 - 15m
    6 == 6 -> 3.6 - 15 m
"""

In [ ]:
"""
Diámetro - BxS - Tiempo entre Pozos Filas ms - Fc - Tipo Explosivo
"""

In [ ]:
FC=list(range(100,1001))

In [ ]:
diametro=[6,6.5,7.875,9.875,10.625,12.25]

In [ ]:
B=list(np.arange(3.5,15.1,0.1))
B = [ round(elem, 2) for elem in B ]

S=list(np.arange(3.5,15.1,0.1))
S = [ round(elem, 2) for elem in S ]

In [ ]:
tiempo_1=list(range(0,201))
tiempo_2=list(range(0,201))

In [ ]:
for i in FC:
    for j in diametro:
        for k in B:
            for l in S:
                for m in tiempo_1:
                    for n in tiempo_2:
                        if m>n: # condiciones de borde
                            continue
                        elif (j == 6     and ( (k<3.5 or k>=15) or (l<3.5 or l>=15) ) ):  # condiciones de borde
                            continue
                        elif (j == 6.5   and ( (k<4 or k>=15)   or (l<4   or l>=15) ) ):  # condiciones de borde
                            continue
                        elif (j == 7.875 and ( (k<4 or k>=15)   or (l<4   or l>=15) ) ):  # condiciones de borde
                            continue
                        elif (j == 9.875 and ( (k<5 or k>=15)   or (l<5   or l>=15) ) ):  # condiciones de borde
                            continue 
                        elif (j == 10.625and ( (k<5.5 or k>=15) or (l<5.5 or l>=15) ) ):  # condiciones de borde
                            continue
                        elif (j == 12.25 and ( (k<5.5 or k>=15) or (l<5.5 or l>=15) ) ):  # condiciones de borde
                            continue
                        else: # cumple con las condiciones minimas
                            print ("FC: " + str(i))
                            print ("diametro: "+ str(j) )
                            print ("B: " + str(k))
                            print ("S: "+ str(l))
                            print ("tiempo_1: "+ str(m))
                            print ("tiempo_2: "+ str(n))
                            